# LLM Agent Capabilities
- List down places where `Harry Potter` movie franchise was shot in England.
- Verify if there is any Famous Cricket Ground nearby, if yes, identify the location as area of interest.
- Plot both locations on a map

## Planning
- What Tools would the agent need?
    - Search API
    - Web Scraping
    - Travel time calculator (custom tool)
    - Plotting tools (map)

In [10]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [11]:
# Setup langfuse client
from langfuse import Langfuse

langfuse = Langfuse(
	secret_key=os.getenv("SECRET_KEY"),
	public_key=os.getenv("PUBLIC_KEY"),
	host=os.getenv("LANGFUSE_HOST"),
)

In [12]:
# authenticate langfuse client connection
from langfuse import get_client

langfuse = get_client()

# Verify connection does not use in production as this is a synchronous call
if langfuse.auth_check():
	print("Langfuse client is authenticated and ready!")
else:
	print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


In [15]:
# setup llm observability
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# set up opentelemetry tracer
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

# set up smolagents instrumentor
SmolagentsInstrumentor().instrument(trace_provider=trace_provider)

In [34]:
# get rest of api key setup
hf_write_token = os.getenv("HF_WRITE_TOKEN")
openai_api_key = os.getenv("OPENAI_API_KEY")
serp_api_key = os.getenv("SERP_API_KEY")  # search api
serper_key = os.getenv("SERPER_KEY")  # search api

In [5]:
import math
from typing import Optional, Tuple
from smolagents import tool

F:\repos\python\The_Architect_Guide_To_LLM_Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# helper function to calculate distance between two points on earth
def to_radians(degrees: float) -> float:
	return degrees * math.pi / 180

In [17]:
# Earth raidus in km
EARTH_RADIUS_KM = 6371.0

In [18]:
# custom tool to identify travel time via chartered flight
@tool
def calculate_air_travel_time(
	origin_coords: Tuple[float, float],
	destination_coords: Tuple[float, float],
	cruising_speed_kmh: Optional[float] = 925,
) -> float:
	"""Calculate the air travel time between two points.
	Args:
	    origin_coords: Tuple of (latitude, longitude) for the origin point.
	    destination_coords: Tuple of (latitude, longitude) for the destination point.
	    cruising_speed_kmh: The cruising speed in km/h. Defaults to 925 for cross-atlantic chartered flights.
	Returns:
	    float: The air travel time in hours.
	"""
	# extract lat long from input args
	lat1, long1 = map(to_radians, origin_coords)
	lat2, long2 = map(to_radians, destination_coords)

	dlong = long2 - long1
	dlat = lat2 - lat1

	a = (
		math.sin(dlat / 2) ** 2
		+ math.cos(lat1) * math.cos(lat2) * math.sin(dlong / 2) ** 2
	)
	c = 2 * math.asin(math.sqrt(a))
	distance = EARTH_RADIUS_KM * c

	# add 5% for air traffic congestion
	final_distance = distance * 1.05

	# calculate air travel time in hours
	air_travel_time_hours = (
		final_distance / cruising_speed_kmh + 1.0
	)  # takeoff and landing procedure
	return round(air_travel_time_hours, 2)

In [39]:
# helper function to get lat long of a location
from geopy.geocoders import Nominatim


@tool
def get_lat_long(location_name: str) -> Optional[Tuple[float, float]]:
	"""Get the latitude and longitude of a location using the Nominatim API.
	Args:
	    location_name (str): The name of the location.
	Returns:
	    Optional[Tuple[float, float]]: A tuple containing the latitude and longitude, or None if the location could not be found.
	"""
	geolocator = Nominatim(user_agent="geopy_example", timeout=10)
	location = geolocator.geocode(location_name)

	if location:
		latitude, longitude = location.latitude, location.longitude
		return latitude, longitude
	else:
		return None

In [20]:
print(f"Location of Ahmedabad is {get_lat_long('Ahmedabad, India')}")
print(f"Location of London is {get_lat_long('London, UK')}")

Location of Ahmedabad is (23.0215374, 72.5800568)
Location of London is (51.4893335, -0.1440551)


In [24]:
# helper function to convert hours into days, hours, and minutes
def convert_hours(total_hours):
	"""
	Converts a given number of hours into days, remaining hours, and minutes.

	Args:
	    total_hours (float or int): The total number of hours to convert.

	Returns:
	    tuple: A tuple containing (days, hours, minutes).
	"""
	# Calculate days
	days = int(total_hours // 24)

	# Calculate remaining hours after extracting days
	remaining_hours = total_hours % 24

	# Calculate whole hours from the remaining_hours
	hours = int(remaining_hours)

	# Calculate minutes from the fractional part of remaining_hours
	minutes = int((remaining_hours - hours) * 60)

	return days, hours, minutes


# Example usage:
hours_input = 75.5
days, hours, minutes = convert_hours(hours_input)
print(
	f"{hours_input} hours is equivalent to {days} days, {hours} hours, and {minutes} minutes."
)

hours_input_2 = 48
days_2, hours_2, minutes_2 = convert_hours(hours_input_2)
print(
	f"{hours_input_2} hours is equivalent to {days_2} days, {hours_2} hours, and {minutes_2} minutes."
)

hours_input_3 = 1.75
days_3, hours_3, minutes_3 = convert_hours(hours_input_3)
print(
	f"{hours_input_3} hours is equivalent to {days_3} days, {hours_3} hours, and {minutes_3} minutes."
)

75.5 hours is equivalent to 3 days, 3 hours, and 30 minutes.
48 hours is equivalent to 2 days, 0 hours, and 0 minutes.
1.75 hours is equivalent to 0 days, 1 hours, and 45 minutes.


In [25]:
# travel time between Ahmedabad and London
travel_time_hours = calculate_air_travel_time(
	(23.0215374, 72.5800568), (51.4893335, -0.1440551)
)
travel_time_print = convert_hours(travel_time_hours)

In [27]:
print(
	f"Air Travel Time between Ahmedabad and London is {travel_time_print[0]} days, {travel_time_print[1]} hours, and {travel_time_print[2]} minutes."
)

Air Travel Time between Ahmedabad and London is 0 days, 8 hours, and 46 minutes.


## now moving to agent creation

In [28]:
from smolagents import (
	CodeAgent,
	GoogleSearchTool,
	InferenceClientModel,
	VisitWebpageTool,
)

In [29]:
model = InferenceClientModel(
	model_id="Qwen/Qwen3-Next-80B-A3B-Instruct",
	provider="together",
	api_key=hf_write_token,
)

In [40]:
task = """Find all Harry Potter filming locations in the world.
Calculate the time to transfer via passenger plane when I am located in Ahmedabad, India located at geo location 23°01′21″N, 72°34′17″E and return result as a pandas dataframe.
Also give me list of famous cricket grounds nearby to the filming locations"""

In [35]:
# Serp api key has to be part of environment, there is no option to pass it as argument
os.environ["SERPAPI_API_KEY"] = serp_api_key

In [44]:
agent = CodeAgent(
	model=model,
	tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_air_travel_time],
	additional_authorized_imports=["pandas"],
	max_steps=20,
)

In [45]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Harry Potter filming locations in the world.                                                           │
│ Calculate the time to transfer via passenger plane when I am located in Ahmedabad, India located at geo         │
│ location 23°01′21″N, 72°34′17″E and return result as a pandas dataframe.                                        │
│ Also give me list of famous cricket grounds nearby to the filming locations                                     │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  harry_potter_locations = web_search(query="all Harry Potter filming locations in the world")                     
  print(harry_potter_locations)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [Harry Potter filming locations guide - VisitBritain 
Shop](https://www.visitbritainshop.com/us/en/harry-potter-filming-locations-guide)
Source: visitbritainshop.com

From the bustling streets of London to the ancient castles of Scotland and the charming countryside of England, 
every pin on this map reveals a new adventure.

1. [Top Sights Where 'Harry Potter' Was 
Filmed](https://www.ricksteves.com/watch-read-listen/read/articles/harry-potter-sites)
Source: Rick Steves Europe

Top Sights Where 'Harry Potter' Was Filmed · London · Near Bath · Near Brighton · Oxford · Durham and Northeast 
England · Scottish Highlands · Ireland.

2. [The Complete Guide to Harry Potter Film Locations ...](https://londonmymind.com/harry-potter-film-locations/)
Date published: May 9, 2023
Source: London On My Mind

Sure, the studios in Leavesden were one of the primary Harry Potter film locations, but they also filmed across 
London, England, the U.K., and ...

3. [A Magical Guide to the Best Harry Potter Places to 
Visit](https://heleneinbetween.com/2025/03/harry-potter-filming-locations-best-places-to-travel-for-potterheads.htm
l)
Source: Helene in Between

If you're a Harry Potter fan, you have to visit the Warner Brothers Studio in London. It's the place where all 8 of
the Harry Potter movies were filmed. You can ...

4. [Where to find all Harry Potter Destinations Around the 
World](https://www.worldofwanderlust.com/find-harry-potter-destinations-around-world/)
Source: World of Wanderlust

One of the locations for filming Hogwarts School of Witchcraft and Wizadry was Durham Cathedral. This is also one 
of the most spectacular filming locations ...

5. [Places in Harry Potter](https://en.wikipedia.org/wiki/Places_in_Harry_Potter)
Source: Wikipedia

Contents · 3.1 The Leaky Cauldron · 3.2 Diagon Alley · 3.3 Knockturn Alley · 3.4 Platform Nine and Three-Quarters ·
3.5 Hogsmeade · 3.6 La Place Cachée, Paris.

6. [UK Harry Potter Filming Locations & 
Attractions](https://www.visitbritain.com/en/things-to-do/visit-harry-potter-filming-locations)
Source: VisitBritain

Harry Potter film locations around Britain · Freshwater West, Pembrokeshire, Wales · Lacock Abbey, Wiltshire, 
England · Alnwick Castle, Northumberland, England.

7. [31 Must Visit Harry Potter Filming Locations in the UK and 
...](https://flavourofthefilm.com/2024/11/01/31-must-visit-harry-potter-filming-locations-in-the-uk-and-ireland/)
Date published: Nov 1, 2024
Source: Flavour of the Film

In this Harry Potter filming locations list, I will detail the exact locations and provide you with information on 
how to get to them

8. [Magical Harry Potter Filming Locations Tours And 
...](https://www.ahoymatey.blog/harry-potter-filming-locations/)
Source: Ahoy Matey Blog

Explore iconic Harry Potter filming locations, tours, and experiences in the UK and USA, from Hogwarts castles to 
Diagon Alley. Step into the magic today!

Out: None

[Step 1: Duration 1.65 seconds| Input tokens: 2,319 | Output tokens: 60]

...

[Step 6: Duration 19.54 seconds| Input tokens: 27,462 | Output tokens: 1,926]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Final answer: Return the DataFrame and the list of cricket grounds                                             
  final_answer({                                                                                                   
      "Air Travel Times from Ahmedabad to Harry Potter Filming Locations":                                         
  df_travel_times.to_dict(orient="records"),                                                                       
      "Famous Cricket Grounds Nearby": cricket_grounds                                                             
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'Air Travel Times from Ahmedabad to Harry Potter Filming Locations': [{'Location': 'Warner Bros. 
Studio Tour London (Leavesden)', 'Travel Time (hours)': 8.79}, {'Location': "King's Cross Station, London", 'Travel
Time (hours)': 8.77}, {'Location': 'Leadenhall Market, London', 'Travel Time (hours)': 8.76}, {'Location': 'Alnwick
Castle, Northumberland', 'Travel Time (hours)': 8.88}, {'Location': 'Lacock Abbey, Wiltshire', 'Travel Time 
(hours)': 8.92}, {'Location': 'Durham Cathedral, Durham', 'Travel Time (hours)': 8.87}, {'Location': 'Freshwater 
West, Pembrokeshire', 'Travel Time (hours)': 9.14}, {'Location': 'Glenfinnan Viaduct, Scottish Highlands', 'Travel 
Time (hours)': 9.14}, {'Location': 'County Wicklow, Ireland', 'Travel Time (hours)': 9.25}], 'Famous Cricket 
Grounds Nearby': [{'Location': 'London', 'Nearby Cricket Grounds': "## Search Results\n0. [List of cricket grounds 
in England and Wales](https://en.wikipedia.org/wiki/List_of_cricket_grounds_in_England_and_Wales)\nDate published: 
Apr 2, 2024\nSource: Wikipedia\n\nHobbs' Ground† (Barclays Bank Ground), Norbury · C. I. Thornton's XI (1888). Kew 
Green · Kew, London (1730–32). Laleham Burway†, near Chertsey, Surrey ...\n\n1. [The 10 Oldest Cricket Grounds In 
The World](https://www.worldatlas.com/articles/the-10-oldest-cricket-grounds-in-the-world.html)\nDate published: 
Oct 17, 2024\nSource: WorldAtlas\n\nLord's Cricket Ground, in St. John's Wood, London, is the oldest cricket venue 
in the world and also one of the oldest sporting stadiums.\n\n2. [Is Lord Cricket Ground Tour worth 
it?](https://www.reddit.com/r/Cricket/comments/1evhxlo/is_lord_cricket_ground_tour_worth_it/)\nSource: Reddit · 
r/Cricket\n\nIts 80 GBP for Lords one (45 GBP for Oval) for the whole family. But we're frugal and every 
museum/palace visit in London adds up.\n\n3. [The 10 most picturesque cricket grounds in 
Britain](https://www.telegraph.co.uk/cricket/0/10-best-picturesque-cricket-grounds-england-wales/)\nDate published:
Mar 26, 2024\nSource: The Telegraph\n\nThe 10 most picturesque cricket grounds in Britain · Abergavenny · Bamburgh 
· Knole Park · Monkton Combe · Stoneleigh · Threlkeld · Valley of the Rocks.\n\n4. [What is the home stadium for 
the English cricket team?](https://www.quora.com/What-is-the-home-stadium-for-the-English-cricket-team)\nSource: 
Quora\n\nLords, The Oval, The Rose Bowl, Edgbaston, Trent Bridge, Old Trafford, Headingly and Chester-le-Street. 
One day matches are played at several other grounds in ...\n\n5. [Rose Bowl (cricket 
ground)](https://en.wikipedia.org/wiki/Rose_Bowl_(cricket_ground))\nDate published: Jan 23, 2024\nSource: 
Wikipedia\n\nThe Rose Bowl, known for sponsorship reasons as Utilita Bowl, is a cricket ground and hotel complex in
West End, Hampshire.\n\n6. [The Oval Cricket Ground is London SE11. My 
friend](https://www.facebook.com/groups/6143110432467287/posts/7623610427750606/)\nSource: Facebook · I grew up in 
South East London\n\n... best cricket ground The Oval London. It is every cricketer's dream to play at such 
prestigious venue. On 21st of September 2024, our club Peper Harrow is ...\n\n7. [Lord's Cricket Ground - Tours & 
Tickets Icon](https://www.stadiumtourtickets.com/en/cricket/london/lords-cricket-ground)\nDate published: Jun 21, 
2024\nSource: stadiumtourtickets.com\n\nHome to the Marylebone Cricket Club (MCC) and steeped in history, Lord's 
offers an immersive stadium tour that promises an unforgettable experience for cricket ...\n\n8. [Cricket Tours: 7 
Unforgettable London 
Experiences](https://londonsightseeingtaxitours.com/cricket-tours-explore-londons-cricketing-heritage/)\nDate 
published: Nov 18, 2024\nSource: London Sightseeing Taxi Tours\n\nVisiting iconic cricket grounds such as Lord's 
Cricket Ground, known as the 'Home of Cricket'; Exploring cricket-related exhibits at the Museum of London."}, 
{'Location': 'Alnwick Castle', 'Nearby Cricket Grounds': "## Search Results\n0. [Bamburgh Castle Cricket

[Step 7: Duration 1.66 seconds| Input tokens: 39,557 | Output tokens: 2,048]